In [1]:
# installing the libraries for transformers
#!pip install -U -q sentence-transformers transformers bitsandbytes accelerate sentencepiece

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Deep Learning library
import torch

# to load transformer models
#from sentence_transformers import SentenceTransformer
#from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline # T5 is Google

# to split the data
from sklearn.model_selection import train_test_split

# to compute performance metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


In [3]:

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        files.append(os.path.join(dirname, filename))
misconceptions_filename = files[1]
train_filename = files[2]
test_filename = files[3]
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eedi-mining-misconceptions-in-mathematics/sample_submission.csv
/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv
/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv
/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv


## File and Field Information
### [train/test].csv

* QuestionId - Unique question identifier (int).
* ConstructId - Unique construct identifier (int) .
* ConstructName - Most granular level of knowledge related to question (str).
* CorrectAnswer - A, B, C or D (char).
* SubjectId - Unique subject identifier (int).
* SubjectName - More general context than the construct (str).
* QuestionText - Question text extracted from the question image using human-in-the-loop OCR (str) .
* Answer[A/B/C/D]Text - Answer option A text extracted from the question image using human-in-the-loop OCR (str).
* Misconception[A/B/C/D]Id - Unique misconception identifier (int). Ground truth labels in train.csv; your task is to predict these labels for test.csv.

### misconception_mapping.csv
maps MisconceptionId to its MisconceptionName

### sample_submission.csv
A submission file in the correct format.
* QuestionId_Answer - Each question has three incorrect answers for which need you predict the MisconceptionId.
* MisconceptionId - You can predict up to 25 values, space delimited.

In [4]:
train_data = pd.read_csv(train_filename)
train_data_orig = train_data.copy()
train_data.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,NaN,NaN,NaN,1672.0
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN
2,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0
3,3,2377,Recall and use the intersecting diagonals prop...,88,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with ...,acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0
4,4,3387,Substitute positive integer values into formul...,67,Substitution into Formula,A,The equation \( f=3 r^{2}+3 \) is used to find...,\( 30 \),\( 27 \),\( 51 \),\( 24 \),NaN,NaN,NaN,1818.0


In [5]:
print ("train_data shape")
print("rows",train_data.shape[0])
print("columns",train_data.shape[1])

train_data shape
rows 1869
columns 15


In [6]:
subject_groups = train_data.groupby(["SubjectId","SubjectName"]).count().sort_values('QuestionId',ascending=False).reset_index()
print(subject_groups.shape[0], " different subject groups")
subject_groups.head()

163  different subject groups


,SubjectId,SubjectName,QuestionId,ConstructId,ConstructName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,64,Linear Equations,53,53,53,53,53,53,53,53,53,23,32,30,27
1,171,Linear Sequences (nth term),44,44,44,44,44,44,44,44,44,28,20,21,19
2,33,BIDMAS,37,37,37,37,37,37,37,37,37,20,21,23,24
3,65,Quadratic Equations,36,36,36,36,36,36,36,36,36,18,17,25,14
4,75,Area of Simple Shapes,36,36,36,36,36,36,36,36,36,22,26,23,24


In [7]:
print("percentages of null values")
pd.DataFrame({'Count':train_data.isnull().sum()[train_data.isnull().sum()>0],'Percentage':(train_data.isnull().sum()[train_data.isnull().sum()>0]/train_data.shape[0])*100})

percentages of null values


,Count,Percentage
MisconceptionAId,734,39.272338
MisconceptionBId,751,40.181915
MisconceptionCId,789,42.215088
MisconceptionDId,832,44.515784


## Clean up missing data

In [8]:
# records where more than one misconception id is missing
missing_a_misconception = train_data[(train_data.CorrectAnswer == "A") & (train_data.MisconceptionBId.isnull() | train_data.MisconceptionCId.isnull()  | train_data.MisconceptionDId.isnull()) ]
print(missing_a_misconception.shape)
missing_a_misconception.head()
train_data[(train_data.CorrectAnswer == "A")].shape

(244, 15)


(482, 15)

In [9]:
missing_b_misconpception = train_data[(train_data.CorrectAnswer == "B") & (train_data.MisconceptionAId.isnull() | train_data.MisconceptionCId.isnull()) ]
missing_b_misconpception.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
10,10,1414,Expand products of three binomials in the form...,335,Expanding Triple Brackets and more,B,John is expanding these three brackets:\n\(\n(...,\( +6 x^{2} \),\( \begin{array}{l}+6 x^{2} \\ -3 x^{2} \\ +5 ...,\( \begin{array}{l}+6 x^{2} \\ +3 x^{2} \\ +5 ...,\( \begin{array}{l}+6 x^{2} \\ -3 x^{2} \\ -5 ...,NaN,NaN,583.0,NaN
13,13,3065,Convert from hours to minutes,209,Time,B,Hannah's journey to camp is \( 3 \) hours and ...,\( 196 \) minutes,\( 18 \) minutes,\( 102 \) minutes,\( 12 \) minutes,NaN,NaN,161.0,NaN
18,18,261,Carry out missing number subtraction problems ...,211,Adding and Subtracting Negative Numbers,B,![Number line with -12 and -7 marked. Starting...,\( -7 \),\( -5 \),\( -2 \),\( -6 \),2179.0,NaN,NaN,1824.0
20,20,2306,"Given the volume of a cuboid, work out missing...",189,Volume of Prisms,B,The volume of this cuboid is \( 30 \mathrm{~cm...,\( 6 \mathrm{~cm} \),\( 5 \mathrm{~cm} \),\( 4 \mathrm{~cm} \),\( 25 \mathrm{~cm} \),NaN,NaN,NaN,1984.0
21,21,1434,Factorise a quadratic expression in the form a...,53,Factorising into a Double Bracket,B,Step 1: Factorise the following expression\n\n...,\( (3 x+2)(3 x+1) \),\( (3 x+2)(x+1) \),Cannot be factorised,\( (3 x+1)(x+2) \),2240.0,NaN,NaN,NaN


In [10]:
misc_con_data = pd.read_csv(misconceptions_filename)
misc_con_data.head()

,MisconceptionId,MisconceptionName
0,0,Does not know that angles in a triangle sum to...
1,1,Uses dividing fractions method for multiplying...
2,2,Believes there are 100 degrees in a full turn
3,3,Thinks a quadratic without a non variable term...
4,4,Believes addition of terms and powers of terms...


## Update data to prepare for creating vectors


In [11]:
train_data['CleanQuestion'] = train_data['QuestionText'].replace('\n','',regex=True)

In [12]:
train_data.head(20)

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId,CleanQuestion
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,NaN,NaN,NaN,1672.0,\[3 \times 2+4-5\]Where do the brackets need t...
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN,"Simplify the following, if possible: \( \frac{..."
2,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0,Tom and Katie are discussing the \( 5 \) plant...
3,3,2377,Recall and use the intersecting diagonals prop...,88,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with ...,acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0,The angles highlighted on this rectangle with ...
4,4,3387,Substitute positive integer values into formul...,67,Substitution into Formula,A,The equation \( f=3 r^{2}+3 \) is used to find...,\( 30 \),\( 27 \),\( 51 \),\( 24 \),NaN,NaN,NaN,1818.0,The equation \( f=3 r^{2}+3 \) is used to find...
5,5,2052,Identify a unit of area,75,Area of Simple Shapes,D,James has answered a question on the area of a...,\( m \),\( \mathrm{cm} \),\( \mathrm{km}^{3} \),\( \mathrm{mm}^{2} \),686.0,686.0,686.0,NaN,James has answered a question on the area of a...
6,6,376,Convert two digit integer percentages to fract...,238,Converting between Fractions and Percentages,B,Convert this percentage to a fraction\n\( 62 \...,\( \frac{62}{10} \),\( \frac{31}{50} \),\( \frac{6}{2} \),None of these,329.0,NaN,847.0,329.0,Convert this percentage to a fraction\( 62 \% \)
7,7,314,Divide decimals by 10,224,Multiplying and Dividing with Decimals,A,\( 43.2 \div 10= \),\( 4.32 \),\( 0.432 \),\( 33.2 \),\( 43.02 \),NaN,2123.0,2273.0,2133.0,\( 43.2 \div 10= \)
8,8,435,Subtract proper fractions with different denom...,230,Adding and Subtracting Fractions,A,\(\n\frac{4}{5}-\frac{1}{3}=\frac{\bigstar}{15...,\( 7 \),\( 5 \),\( 17 \),\( 3 \),NaN,907.0,1514.0,907.0,\(\frac{4}{5}-\frac{1}{3}=\frac{\bigstar}{15}\...
9,9,1321,Identify horizontal translations in the form f...,164,Transformations of functions in the form f(x),C,What transformation maps the graph of\n\(y=f(x...,Translation by vector\n\(\n\left[\begin{array}...,Translation by vector\n\(\n\left[\begin{array}...,Translation by vector\n\(\n\left[\begin{array}...,Translation by vector\n\(\n\left[\begin{array}...,1889.0,1234.0,NaN,1312.0,What transformation maps the graph of\(y=f(x)\...


In [13]:
# defining a function to compute the cosine similarity between two embedding vectors
def cosine_score(text):
    # encoding the text
    embeddings = model.encode(text)

    # calculating the L2 norm of the embedding vector
    norm1 = np.linalg.norm(embeddings[0])
    norm2 = np.linalg.norm(embeddings[1])

    # computing the cosine similarity
    cosine_similarity_score = ((np.dot(embeddings[0],embeddings[1]))/(norm1*norm2))

    return cosine_similarity_score

In [14]:
# use pylatexenc to convert latex to text then vector